In [1]:
import textwrap # for print long text in wrapped lines
from pprint import pprint # for print pretty json

def format_text(text, width=80, indent=0):
    # Split text into lines
    lines = text.split('\n')
    # Format each line, keeping short lines unchanged
    formatted_lines = []
    for line in lines:
        if len(line) <= width - indent:
            # Keep short lines as is, just add indent
            formatted_lines.append(" "*indent + line)
        else:
            # Wrap long lines
            wrapped = textwrap.fill(line, width=width, initial_indent=" "*indent, subsequent_indent=" "*indent)
            formatted_lines.append(wrapped)
    return '\n'.join(formatted_lines)

## Get embeddings and vector store, do semantic search

In [2]:
import getpass
import os

# for convert text to embedding vectors, so that we can use it for similarity search and RAG
from langchain_openai import OpenAIEmbeddings 

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# from langchain_ollama import OllamaEmbeddings # alternatively use open source models on your own server
# embeddings = OllamaEmbeddings(model="qwen2.5")

from langchain_unstructured import UnstructuredLoader # for load data from unstructured files 
from langchain_text_splitters import RecursiveCharacterTextSplitter # for split text into chunks

In [ ]:
# to use unstructured, we need to set it up following: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file/

file_paths = [
  "../hardware/datasheets/TB6612FNG.pdf"
]
loader = UnstructuredLoader(
  file_paths,
  chunking_strategy="basic",
  max_characters=1000000,
  include_orig_elements=False,
  )


docs = loader.load()
docs[0]

pprint(docs[0].metadata)

print("Number of LangChain documents:", len(docs))
print("Length of text in the document:", len(docs[0].page_content))

In [ ]:
print(format_text(docs[0].page_content[:200]))


In [ ]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)


In [ ]:
print(format_text(all_splits[2].page_content))


In [ ]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
ids = vector_store.add_documents(documents=all_splits)

In [ ]:
# an example of similarity search fo a question
results = vector_store.similarity_search(
    "What is the function of pin AO1?"
)

print("Number of retrieved results:", len(results))
# print the first result
print(format_text(results[0].page_content))

In [ ]:
print(format_text(results[1].page_content))

In [11]:
import getpass
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
# llm = ChatOpenAI(model="o1-preview") # much stronger reasoning

## Build the RAG with LangGraph

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt") # details in https://smith.langchain.com/hub/rlm/rag-prompt


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
response = graph.invoke({"question": f"Output the pin mapping of TB6612FNG in json format, including the pin name, number, and function."})

# print(f'Context: {response["context"]}\n\n')
print(format_text(response["answer"]))

In [ ]:
# an example of using the RAG for question answering

response = graph.invoke({"question": "What does this datasheet say about how to control the motor?"})

print(f'Context: {response["context"]}\n\n')
print(format_text(response["answer"]))

In [ ]:
print(response["answer"])
summary = response["answer"]
# print(textwrap.fill(response["answer"], width=80))

In [ ]:
response = graph.invoke({"question": f"Give this code {summary}, for Arduino Nano, output the C code for the PWM driver, make sure the PWM driver has frequency of 5000 Hz."})

# print(f'Context: {response["context"]}\n\n')
print(format_text(response["answer"]))


In [ ]:
sample_code = response["answer"]
response = graph.invoke({"question": f"Give original code {sample_code}, for Arduino Nano, justify values you used to make sure the PWM driver has frequency of 5000 Hz."})

# print(f'Context: {response["context"]}\n\n')
print(format_text(response["answer"]))

In [ ]:
for step in graph.stream(
    {"question": "Output the pin mapping of TB6612FNG in json format, including the pin name, number, and function."}, stream_mode="updates"
):
    print(f"{step}\n\n----------------\n")